In [1]:
!pip install tqdm
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 62.0 MB/s eta 0:00:00


In [2]:
import os
import rasterio
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict
import calendar
from tqdm import tqdm 
import re
from osgeo import gdal

# Extract AWS

In [ ]:
def extract_datetime_from_filename(filename):
    # Trường hợp 1: Định dạng B04B_20201016.Z2100_TB.tif
    match1 = re.search(r'^(.*?)_(\d{8})\.Z(\d{4})_TB', filename)
    
    if match1:
        date_part, time_part = match1.group(2), match1.group(3)
        dt_str = f"{date_part}{time_part}00"  # Ghép thành YYYYMMDDhhmm00
        
        try:
            return datetime.strptime(dt_str, "%Y%m%d%H%M%S")
        except ValueError:
            return None

    # Trường hợp 2: Định dạng AWS_20190420000000.tif
    match2 = re.search(r'^(.*?)_(\d{14})\.tif$', filename)

    if match2:
        dt_str = match2.group(2)  # Lấy chuỗi YYYYMMDDHHMMSS
        
        try:
            return datetime.strptime(dt_str, "%Y%m%d%H%M%S")
        except ValueError:
            return None

    return None

In [ ]:
def extract_data(root_dir, option=1, coordinates=None):
    """
    Trích xuất dữ liệu từ các file GeoTIFF trong thư mục root_dir và bổ sung dữ liệu bị thiếu.
    
    - root_dir: đường dẫn thư mục gốc
    - option: 
        + 1 - Trả về toàn bộ dữ liệu
        + 2 - Chỉ lấy dữ liệu ở các tọa độ cụ thể (row, col) trong tập `coordinates`
    - coordinates: set chứa các tọa độ (row, col) nếu option = 2
    
    Returns:
    - dict chứa DataFrame của từng tháng, với key là "YYYY-MM".
    """
    
    monthly_data = {}
    param_name = root_dir.split('/')[-1]  # Tự động lấy tên biến khí tượng

    for year in ['2019', '2020']:
        year_path = os.path.join(root_dir, year)
        if not os.path.isdir(year_path):
            continue

        for month in sorted(os.listdir(year_path)):
            month_path = os.path.join(year_path, month)
            if not os.path.isdir(month_path):
                continue

            available_timestamps = set()
            pixel_values = {}  # Lưu dữ liệu của từng pixel

            for day in tqdm(sorted(os.listdir(month_path)), desc=f"Đang xử lý {year}/{month}"):
                day_path = os.path.join(month_path, day)
                if not os.path.isdir(day_path):
                    continue

                for file in sorted(os.listdir(day_path)):
                    if not file.endswith(".tif"):
                        continue
                    file_path = os.path.join(day_path, file)

                    dataset = gdal.Open(file_path)
                    if dataset is None:
                        print(f" Không mở được file: {file_path}")
                        continue

                    band = dataset.GetRasterBand(1)
                    data = band.ReadAsArray()

                    # Xử lý tên file để lấy datetime
                    dt = extract_datetime_from_filename(file)

                    if dt is None:
                        print(f" Lỗi xử lý datetime cho file: {file}")
                        continue
                    
                    available_timestamps.add(dt)

                    rows, cols = data.shape
                    row_idx, col_idx = np.indices((rows, cols))

                    # Áp dụng điều kiện lọc dữ liệu hợp lệ
                    valid_mask = np.ones_like(data, dtype=bool)

                    # Nếu option == 2, chỉ giữ lại các tọa độ có trong `coordinates`
                    if option == 2 and coordinates:
                        coord_mask = np.vectorize(lambda r, c: (r, c) in coordinates)(row_idx, col_idx)
                        valid_mask &= coord_mask

                    values = data[valid_mask]
                    row_idx = row_idx[valid_mask]
                    col_idx = col_idx[valid_mask]

                    for r, c, v in zip(row_idx, col_idx, values):
                        if (r, c) not in pixel_values:
                            pixel_values[(r, c)] = {}
                        pixel_values[(r, c)][dt] = v

            # Tạo danh sách timestamp đầy đủ cho tháng đó
            start_date = datetime(int(year), int(month), 1, 0, 0, 0)
            num_days_in_month = calendar.monthrange(int(year), int(month))[1]
            end_date = datetime(int(year), int(month), num_days_in_month, 23, 59, 59)
            all_timestamps = {start_date + timedelta(hours=i) for i in range((end_date - start_date).days * 24 + 24)}

            missing_timestamps = all_timestamps - available_timestamps

            # Điền NaN vào các timestamp bị thiếu
            for pixel, values in pixel_values.items():
                for ts in missing_timestamps:
                    values[ts] = np.nan

            # Chuyển dữ liệu thành DataFrame
            data_list = []
            for (r, c), values in pixel_values.items():
                for ts, v in values.items():
                    data_list.append([ts, r, c, v])

            month_key = f"{year}-{month}"
            df = pd.DataFrame(data_list, columns=["datetime", "row", "col", param_name])
            df["datetime"] = pd.to_datetime(df["datetime"])
            df["row"] = df["row"].astype(int)
            df["col"] = df["col"].astype(int)
            df[param_name] = df[param_name].astype(float)

            monthly_data[month_key] = df

    return monthly_data

In [6]:
# AWS raw data
aws_raw_data = extract_data('/kaggle/input/dataai/DATA_SV/Precipitation/AWS')
aws_raw_data.keys()

Đang xử lý 2020/10: 100%|██████████| 31/31 [00:26<00:00,  1.19it/s]


dict_keys(['2019-04', '2019-10', '2020-04', '2020-10'])

In [7]:
def get_valid_pixels(result):
    """
    Xác định các pixel có ít nhất một giá trị hợp lệ trong bất kỳ tháng nào.
    """
    if not result:
        return set()

    valid_pixels = set()
    param_name = next(iter(result.values())).columns[-1]  # Lấy tên cột giá trị khí tượng

    for df in tqdm(result.values()):
        valid_pixels.update(
            (row, col) for row, col, value in zip(df["row"], df["col"], df[param_name])
            if value != -np.inf and not pd.isna(value)
        )

    return valid_pixels

In [8]:
aws_valid_pixels = get_valid_pixels(aws_raw_data)

len(aws_valid_pixels)

100%|██████████| 4/4 [00:24<00:00,  6.05s/it]


334

In [9]:
filtered_aws_data = extract_data('/kaggle/input/dataai/DATA_SV/Precipitation/AWS', option = 2, coordinates=aws_valid_pixels)

Đang xử lý 2020/10: 100%|██████████| 31/31 [00:05<00:00,  6.04it/s]


In [10]:
for key, value in filtered_aws_data.items():
    print(key)
    print(value.info())

2019-04
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240480 entries, 0 to 240479
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   datetime  240480 non-null  datetime64[ns]
 1   row       240480 non-null  int64         
 2   col       240480 non-null  int64         
 3   AWS       240480 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 7.3 MB
None
2019-10
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248496 entries, 0 to 248495
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   datetime  248496 non-null  datetime64[ns]
 1   row       248496 non-null  int64         
 2   col       248496 non-null  int64         
 3   AWS       209418 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 7.6 MB
None
2020-04
<class 'pandas.core.frame.DataFrame'>
RangeI

# Extract ERA5

# Extract Himawari

In [11]:
hima_root = "/kaggle/input/dataai/DATA_SV/Hima"

hima_data_dict = {}

for subdir in sorted(os.listdir(hima_root)):
    subdir_path = os.path.join(hima_root, subdir)
    
    if os.path.isdir(subdir_path): 
        print(f" Đang xử lý folder con: {subdir}")
        hima_data_dict[subdir] = extract_data(subdir_path, option=2, coordinates=aws_valid_pixels)

 Đang xử lý folder con: B04B


Đang xử lý 2020/10: 100%|██████████| 31/31 [00:06<00:00,  5.08it/s]


 Đang xử lý folder con: B05B


Đang xử lý 2020/10: 100%|██████████| 31/31 [00:05<00:00,  5.57it/s]


 Đang xử lý folder con: B06B


...

Đang xử lý 2020/10: 100%|██████████| 31/31 [00:13<00:00,  2.28it/s]


 Đang xử lý folder con: VSB


Đang xử lý 2020/10: 100%|██████████| 31/31 [00:06<00:00,  4.79it/s]


 Đang xử lý folder con: WVB


Đang xử lý 2020/10: 100%|██████████| 31/31 [00:14<00:00,  2.21it/s]


In [12]:
era5_root = "/kaggle/input/dataai/DATA_SV/ERA5"
era5_data_dict = {}

for subdir in sorted(os.listdir(era5_root)):
    subdir_path = os.path.join(era5_root, subdir)
    
    if os.path.isdir(subdir_path): 
        print(f" Đang xử lý folder con: {subdir}")
        era5_data_dict[subdir] = extract_data(subdir_path, option=2, coordinates=aws_valid_pixels)

 Đang xử lý folder con: CAPE


Đang xử lý 2020/10: 100%|██████████| 31/31 [00:11<00:00,  2.81it/s]


 Đang xử lý folder con: CIN


Đang xử lý 2020/10: 100%|██████████| 31/31 [00:09<00:00,  3.18it/s]


 Đang xử lý folder con: EWSS


...

Đang xử lý 2020/10: 100%|██████████| 31/31 [00:11<00:00,  2.66it/s]


 Đang xử lý folder con: V250


Đang xử lý 2020/10: 100%|██████████| 31/31 [00:11<00:00,  2.69it/s]


 Đang xử lý folder con: V850


Đang xử lý 2020/10: 100%|██████████| 31/31 [00:11<00:00,  2.69it/s]


In [13]:
print(era5_data_dict.keys()) 

dict_keys(['CAPE', 'CIN', 'EWSS', 'IE', 'ISOR', 'KX', 'PEV', 'R250', 'R500', 'R850', 'SLHF', 'SLOR', 'SSHF', 'TCLW', 'TCW', 'TCWV', 'U250', 'U850', 'V250', 'V850'])


In [14]:
print(hima_data_dict.keys())

dict_keys(['B04B', 'B05B', 'B06B', 'B09B', 'B10B', 'B11B', 'B12B', 'B14B', 'B16B', 'I2B', 'I4B', 'IRB', 'VSB', 'WVB'])


# Merge Data

In [15]:
output_dir = "/kaggle/working/merged_data"
os.makedirs(output_dir, exist_ok=True)

for month in filtered_aws_data.keys():
    df = filtered_aws_data[month]

    # Merge với Himawari
    for band_name, band_dict in hima_data_dict.items():
        if month in band_dict:
            df = df.merge(band_dict[month], on=["datetime", "row", "col"], how="left", suffixes=("", f"_{band_name}"))
    
    # Merge với ERA5
    for var_name, era5_dict in era5_data_dict.items():
        if month in era5_dict:
            df = df.merge(era5_dict[month], on=["datetime", "row", "col"], how="left", suffixes=("", f"_{var_name}"))

    output_path = os.path.join(output_dir, f"merged_{month}.csv")
    df.to_csv(output_path, index=False)
    print(f"Đã lưu {output_path}")

print("Done!")


Đã lưu /kaggle/working/merged_data/merged_2019-04.csv
Đã lưu /kaggle/working/merged_data/merged_2019-10.csv
Đã lưu /kaggle/working/merged_data/merged_2020-04.csv
Đã lưu /kaggle/working/merged_data/merged_2020-10.csv
Done!
